<a href="https://colab.research.google.com/github/choiwonjini/AARRR/blob/main/AARRR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 데이터 확인

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/content/drive/MyDrive/Colab Notebooks/AARRR/aarrr-project-3f905304d852.json"

# Client 객체 생성
client = bigquery.Client()

# 데이터셋 참조경로(reference) 설정
dataset_ref = client.dataset('google_analytics_sample', project='bigquery-public-data')

# 해당 경로로부터 데이터셋 추출
dataset = client.get_dataset(dataset_ref)

# 데이터셋을 테이블 단위로 보기
tables = list(client.list_tables(dataset))
table_names = sorted([t.table_id for t in tables])

# 테이블 단위로 간단한 정보 확인
print(f"""table 개수 : {len(tables)}
tables : {", ".join(table_names[:3])}, ...
date 범위 : {table_names[0][-8:]} ~ {table_names[-1][-8:]}""")

table 개수 : 366
tables : ga_sessions_20160801, ga_sessions_20160802, ga_sessions_20160803, ...
date 범위 : 20160801 ~ 20170801


In [ ]:
# 테이블 경로 생성
table_ref_temp = dataset_ref.table(table_names[0])

# 테이블 가져오기
table_temp = client.get_table(table_ref_temp)

# 컬럼 확인
client.list_rows(table_temp, max_results=5).to_dataframe()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,None,Organic Search,Not Socially Engaged
1,<NA>,1,1470084717,1470084717,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 18, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0288478011259077136,None,Direct,Not Socially Engaged
2,<NA>,3,1470078988,1470078988,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6440789996634275026,None,Organic Search,Not Socially Engaged
3,<NA>,4,1470075581,1470075581,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 19, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",8520115029387302083,None,Referral,Not Socially Engaged
4,<NA>,30,1470099026,1470099026,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 17, 'minu...",6792260745822342947,None,Organic Search,Not Socially Engaged


* 아래는 위의 구조체 컬럼 내에 어떤 데이터가 포함되었는지 확인하는 과정

In [ ]:
def format_schema_field(schema_field, indent=0):
    """
    빅쿼리 스키마의 (중첩된 구조 내부까지) 필드 이름과 데이터 타입을 출력하는 함수
    """

    indent_str = "  " * indent
    field_info = f"{indent_str}{schema_field.name} ({schema_field.field_type})"

    if schema_field.mode != "NULLABLE":
        field_info += f" - {schema_field.mode}"

    if schema_field.description:
        field_info += f" - {schema_field.description}"

    nested_indent = indent + 2
    if schema_field.field_type == "RECORD":
        for sub_field in schema_field.fields:
            field_info += "\n" + format_schema_field(sub_field, nested_indent)

    return field_info

# Display schemas
print("SCHEMA field for the 'totals' column:\n")
print(format_schema_field(table_temp.schema[5]))
print()

print("\nSCHEMA field for the 'trafficSource' column:\n")
print(format_schema_field(table_temp.schema[6]))
print()


print("\nSCHEMA field for the 'device' column:\n")
print(format_schema_field(table_temp.schema[7]))
print()

print("\nSCHEMA field for the 'geoNetwork' column:\n")
print(format_schema_field(table_temp.schema[8]))
print()

print("\nSCHEMA field for the 'customDimensions' column:\n")
print(format_schema_field(table_temp.schema[9]))
print()

print("\nSCHEMA field for the 'hits' column:\n")
print(format_schema_field(table_temp.schema[10]))

SCHEMA field for the 'totals' column:

totals (RECORD)
    visits (INTEGER)
    hits (INTEGER)
    pageviews (INTEGER)
    timeOnSite (INTEGER)
    bounces (INTEGER)
    transactions (INTEGER)
    transactionRevenue (INTEGER)
    newVisits (INTEGER)
    screenviews (INTEGER)
    uniqueScreenviews (INTEGER)
    timeOnScreen (INTEGER)
    totalTransactionRevenue (INTEGER)
    sessionQualityDim (INTEGER)


SCHEMA field for the 'trafficSource' column:

trafficSource (RECORD)
    referralPath (STRING)
    campaign (STRING)
    source (STRING)
    medium (STRING)
    keyword (STRING)
    adContent (STRING)
    adwordsClickInfo (RECORD)
        campaignId (INTEGER)
        adGroupId (INTEGER)
        creativeId (INTEGER)
        criteriaId (INTEGER)
        page (INTEGER)
        slot (STRING)
        criteriaParameters (STRING)
        gclId (STRING)
        customerId (INTEGER)
        adNetworkType (STRING)
        targetingCriteria (RECORD)
            boomUserlistId (INTEGER)
        isV

# 1. Acquisition (획득)

**의미**
* 고객이 제품이나 서비스를 처음 접하게 되는 단계
<br>

**목표**
* 다양한 채널(검색, SNS, 광고, 콘텐츠 마케팅 등)을 통해 방문자나 잠재 고객을 유입시키는 것
<br>

**KPI**
* 방문자 수

## (1) 유입 경로

최다 유입 경로 파악

In [ ]:
# 유입이 가장 많이 된 경로 탑 10 쿼리 작성
query = f"""
SELECT
    trafficSource.source AS source,
    count(*) AS visit_count
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
GROUP BY
    source
ORDER BY
    visit_count DESC
LIMIT 10
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_inflow = query_job.to_dataframe()

# 결과 출력
df_inflow

,source,visit_count
0,(direct),371469
1,google,241280
2,youtube.com,212602
3,Partners,16411
4,analytics.google.com,16172
5,dfa,5686
6,google.com,4669
7,m.facebook.com,3365
8,baidu,3356
9,sites.google.com,2983


월별 유입경로

In [ ]:
# 월별 유입 경로 분석 쿼리 작성
query = """
WITH
grouped AS (
  SELECT
      SUBSTR(_TABLE_SUFFIX, 1, 6) AS month,
      trafficSource.source AS source,
      COUNT(*) AS visit_count
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      SUBSTR(_TABLE_SUFFIX, 1, 6), trafficSource.source
),
ranked AS (
  SELECT
      month,
      source,
      visit_count,
      ROW_NUMBER() OVER (
        PARTITION BY month
        ORDER BY visit_count DESC
      ) AS rn
  FROM
      grouped
)
SELECT
    month,
    source,
    visit_count
FROM
    ranked
WHERE
    rn <= 5
ORDER BY
    month, visit_count DESC;
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_monthly = query_job.to_dataframe()

# 결과 출력
df_monthly

,month,source,visit_count
0,201608,google,25170
1,201608,youtube.com,24390
2,201608,(direct),18800
3,201608,Partners,1910
4,201608,analytics.google.com,821
5,201609,youtube.com,23982
6,201609,(direct),21109
7,201609,google,20196
8,201609,Partners,1352
9,201609,analytics.google.com,1082


In [ ]:
##################
# 유입 경로별 신규 방문자, 재방문자 비율 분석 쿼리 작성
query = """
  SELECT
      SUBSTR(_TABLE_SUFFIX, 1, 6) AS month,
      trafficSource.source AS source,
      COUNT(*) AS visit_count
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      SUBSTR(_TABLE_SUFFIX, 1, 6), trafficSource.source
  ORDER BY
      SUBSTR(_TABLE_SUFFIX, 1, 6)
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_11 = query_job.to_dataframe()

# 결과 출력
df_11

df_11.to_csv('월별유입경로.csv', index=False)

In [ ]:
# 유입 경로별 신규 방문자, 재방문자 비율 분석 쿼리 작성
query = """
  SELECT
      SUBSTR(_TABLE_SUFFIX, 1, 6) AS month,
      trafficSource.medium AS medium,
      COUNT(*) AS visit_count
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      SUBSTR(_TABLE_SUFFIX, 1, 6), trafficSource.medium
  ORDER BY
      SUBSTR(_TABLE_SUFFIX, 1, 6)
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_12 = query_job.to_dataframe()

df_12.to_csv('월별방문매체.csv', index=False)

# 결과 출력
df_12

,month,medium,visit_count
0,201608,(none),18800
1,201608,affiliate,1909
2,201608,organic,24143
3,201608,(not set),2
4,201608,cpc,1467
...,...,...,...
74,201707,referral,11053
75,201707,affiliate,1788
76,201707,(none),19890
77,201707,cpm,687


In [ ]:
# 유입 경로별 신규 방문자, 재방문자 비율 분석 쿼리 작성
query = """
  SELECT
      SUBSTR(_TABLE_SUFFIX, 1, 6) AS month,
      trafficSource.campaign AS campaign,
      COUNT(*) AS visit_count
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      SUBSTR(_TABLE_SUFFIX, 1, 6), trafficSource.campaign
  ORDER BY
      SUBSTR(_TABLE_SUFFIX, 1, 6)
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_13 = query_job.to_dataframe()

df_13.to_csv('월별캠페인.csv', index=False)

# 결과 출력
df_13

,month,campaign,visit_count
0,201608,AW - Accessories,8
1,201608,AW - Electronics,46
2,201608,All Products,2
3,201608,AW - Dynamic Search Ads Whole Site,1156
4,201608,Data Share Promo,1909
5,201608,(not set),71638
6,201609,(not set),68566
7,201609,Data Share Promo,1352
8,201609,AW - Accessories,1
9,201609,AW - Dynamic Search Ads Whole Site,1088


각 유입 경로별 신규 방문자와 재방문자 비율

* totals.newVisit: 신규 방문자 = 1, 재방문자 = NULL

In [ ]:
# 유입 경로별 신규 방문자, 재방문자 비율 분석 쿼리 작성
query = """
SELECT
    trafficSource.source AS source,
    COUNT(*) AS visit_count,
    SUM(CASE WHEN totals.newVisits = 1 THEN 1 ELSE 0 END) / COUNT(*) AS new_visitors_rate,
    SUM(CASE WHEN totals.newVisits = 1 THEN 0 ELSE 1 END) / COUNT(*) AS returning_visitors_rate
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
GROUP BY
    source
ORDER BY
    visit_count DESC
LIMIT 10
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_visit = query_job.to_dataframe()

# 결과 출력
df_visit

#df_visit.to_csv('new_re_visit.csv', index=False)

,source,visit_count,new_visitors_rate,returning_visitors_rate
0,(direct),369303,0.705150,0.294850
1,google,241280,0.780815,0.219185
2,youtube.com,212422,0.940340,0.059660
3,Partners,16359,0.762394,0.237606
4,analytics.google.com,16115,0.572076,0.427924
5,dfa,5671,0.269794,0.730206
6,google.com,4657,0.879966,0.120034
7,m.facebook.com,3360,0.902381,0.097619
8,baidu,3351,0.928976,0.071024
9,sites.google.com,2975,0.354958,0.645042


유입 경로별 체류 시간
* 정규분포를 따르지 않고 이상치가 존재
* 그러므로 평균이 아닌 중앙값 사용
* totals.timeOnScreen: 사용자가 웹사이트에서 머문 총 시간을 초 단위로 나타낸 변수


In [ ]:
# 유입 경로별 체류 시간 분석 쿼리 작성
query = """
SELECT
    trafficSource.source AS source,
    COUNT(*) AS visit_count,
    APPROX_QUANTILES(totals.timeOnSite, 100)[SAFE_ORDINAL(50)] AS median_sreen_time
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
GROUP BY
    source
ORDER BY
    visit_count DESC
LIMIT 11
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_timeOnScreen = query_job.to_dataframe()

# 결과 출력
df_timeOnScreen

,source,visit_count,median_sreen_time
0,(direct),371469,99
1,google,241280,95
2,youtube.com,212602,52
3,Partners,16411,105
4,analytics.google.com,16172,74
5,dfa,5686,98
6,google.com,4669,98
7,m.facebook.com,3365,66
8,baidu,3356,57
9,sites.google.com,2983,57


## (2) 캠페인

캠페인별 세션 수

In [ ]:
# 캠페인별 세션 수 분석 쿼리 작성
query = """
SELECT
    trafficSource.campaign AS campaign,
    COUNT(*) AS visit_count,
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
GROUP BY
    campaign
ORDER BY
    visit_count DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_campaign = query_job.to_dataframe()

# 결과 출력
df_campaign

,campaign,visit_count
0,(not set),875599
1,Data Share Promo,16403
2,AW - Dynamic Search Ads Whole Site,6213
3,AW - Accessories,5327
4,AW - Electronics,74
5,AW - Apparel,32
6,All Products,4
7,Data Share,1


## (3) 방문 매체

 방문 매체별 세션 수

In [ ]:
# 방문 매체별 세션 수 분석 쿼리 작성
query = """
SELECT
    trafficSource.medium AS medium,
    COUNT(*) AS visit_count,
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
GROUP BY
    medium
ORDER BY
    visit_count DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_medium = query_job.to_dataframe()

df_medium.to_csv('medium.csv', index=False)

# 결과 출력
df_medium

,medium,visit_count
0,(none),371467
1,referral,262022
2,organic,234381
3,affiliate,16403
4,cpc,13079
5,cpm,6184
6,(not set),117


아래는 Google Analytics의 trafficSource.medium에 나타나는 각 값의 일반적인 의미입니다:
<br>
<br>
none

* 보통 직접 방문(Direct Traffic)을 의미합니다.
* 사용자가 URL을 직접 입력하거나 즐겨찾기를 통해 접속하는 경우, 캠페인 정보 없이 medium이 "(none)"으로 표시됩니다.

referral

* 다른 웹사이트에서 링크를 통해 유입된 트래픽을 의미합니다.
* 방문자가 외부 사이트의 링크를 클릭해 도착한 경우 medium이 "referral"로 설정됩니다.

organic

* 유료 광고가 아닌, 검색엔진의 자연 검색 결과(Organic Search)에서 유입된 트래픽을 의미합니다.
* 구글, 네이버, 빙 등에서 자연 검색 결과로 방문했을 때 medium은 "organic"입니다.

affiliate

* 제휴(affiliate) 마케팅을 통해 유입된 트래픽을 나타냅니다.
* 제휴 프로그램 링크를 통해 방문한 경우 이 값이 사용됩니다.

cpc

* 클릭당 비용(Cost Per Click) 광고를 통해 유입된 트래픽을 의미합니다.
* 구글 애즈와 같은 유료 검색 캠페인에서 클릭을 통해 방문한 경우 medium은 "cpc"입니다.

cpm

* 천 회 노출당 비용(Cost Per Mille) 광고를 통해 유입된 트래픽을 의미합니다.
* 노출 기반의 광고 캠페인에서 유입된 트래픽의 경우 "cpm"이 사용될 수 있습니다.

not set

* 캠페인 매개변수나 추적 정보가 제대로 전달되지 않아, medium 값이 설정되지 않은 경우에 나타납니다.
* 설정 오류나 데이터 누락 등으로 인해 "not set"으로 표시됩니다.

## (4) 국가

세션 수 상위 10개국

In [ ]:
# 국가별 세션 수 분석 쿼리 작성
query = """
SELECT
    geoNetwork.country AS country,
    COUNT(*) AS visit_count,
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
GROUP BY
    country
ORDER BY
    visit_count DESC
LIMIT 10
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_country = query_job.to_dataframe()

df_country.to_csv('country.csv', index=False)

# 결과 출력
df_country

,country,visit_count
0,United States,364744
1,India,51140
2,United Kingdom,37393
3,Canada,25869
4,Vietnam,24598
5,Turkey,20522
6,Thailand,20123
7,Germany,19980
8,Brazil,19783
9,Japan,19731


# 2. Activation (활성화)

**의미**
* 고객이 처음으로 서비스를 사용하며 긍정적인 경험을 하도록 만드는 단계입니다.
* 간단한 가입, 첫 사용, 온보딩(신규 입사자나 사용자가 조직이나 제품에 적응할 수 있도록 지원하는 과정) 완료 등이 해당
<br>

**목표**
* 초기 사용 경험을 통해 고객이 서비스의 가치를 인지하고, ‘즐거운 첫 경험’으로 이어지도록 하는 것
<br>

**KPI**
* 전환율(Conversion Rate)
* 이탈율(Bounce rate)


## 퍼널 분석 (Funnel Analysis)

### 각 단계별 세션 수 산출

hits.eCommerceAction.action_type <br>

1: 제품 목록 <br>
2: 제품 세부정보 보기<br>
3: 장바구니에 제품 추가 <br>
5: 결제 페이지 <br>
6: 구매 완료 페이지 <br>

그 외 0,4,7,8은 각각 알수없음, 장바구니 삭제, 구매환불, 결제옵션

In [ ]:
# 단계별 세션 수 분류 쿼리 작성
query = f"""
SELECT
    h.eCommerceAction.action_type AS action,
    COUNT(*) AS count
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS h
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND h.eCommerceAction.action_type IN ('1', '2', '3', '5', '6')
GROUP BY
    action
ORDER BY
    count DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_funnels = query_job.to_dataframe()

# action 명칭 수정
df_funnels['action'] = ['Product list', 'Product detail page', 'Add to cart', 'Check out', 'Completed purchase']

df_funnels.to_csv('funnels.csv', index=False)

# 결과 출력
df_funnels

,action,count
0,Product list,445760
1,Product detail page,362607
2,Add to cart,121427
3,Check out,88019
4,Completed purchase,24532


### 전환율(Conversion Rate), 이탈률(Bounce rate)

In [ ]:
conversion_rate = [100]

for i in range(0,4):
  conversion_rate.append(round((df_funnels['count'].iloc[i+1]/df_funnels['count'].iloc[i])*100, 2))

df_funnels['conversion rate'] = conversion_rate
df_funnels['Bounce rate'] = np.repeat(100, 5) - np.array(conversion_rate)

df_funnels.to_csv('funnels_con_bounce.csv', index=False)

df_funnels

,action,count,conversion rate,Bounce rate
0,Product list,445760,100.00,0.00
1,Product detail page,362607,81.35,18.65
2,Add to cart,121427,33.49,66.51
3,Check out,88019,72.49,27.51
4,Completed purchase,24532,27.87,72.13


* 병목구간 찾고 개선점 도출

# 3. Retention (유지)

**의미**
* 고객이 지속적으로 서비스를 사용하도록 만드는 단계
* 단발성이 아닌 반복적 사용을 통해 고객과의 관계를 강화
<br>

**목표**
* 고객 만족도를 높여 재방문 및 지속적인 사용을 유도하고, 장기 고객으로 전환시키는 것
<br>

**KPI**
* 전환율(Conversion Rate)
* 이탈율(Bounce rate)

## 코호트 분석 (Cohort Analysis)

* 최초 유입 시점을 기준으로 사용자를 그룹화한 뒤, 일정 기간 동안 얼마만큼의 사용자가 다시 방문하는지를 추적
* 채널별 코호트 비교를 통해 어떤 유입 경로가 장기적으로 더 효과적인지 분석

In [ ]:
# 첫 구매 월과 마지막 구매 월의 차이 기준으로 고객 카운팅 쿼리 작성
query = f"""
WITH purchase_sessions AS (
  SELECT
      fullVisitorId,
      PARSE_DATE('%Y%m', SUBSTR(_TABLE_SUFFIX, 1, 6)) AS session_month
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
      AND totals.transactions > 0  -- 실제 구매 발생한 세션만 고려
),
first_last_purchase AS (
  SELECT
      fullVisitorId,
      MIN(session_month) AS first_purchase_month,
      MAX(session_month) AS last_purchase_month,
      DATE_DIFF(MAX(session_month), MIN(session_month), MONTH) AS month_passed
  FROM
      purchase_sessions
  WHERE
      session_month IS NOT NULL
  GROUP BY
      fullVisitorId
)
SELECT
    FORMAT_DATE('%Y-%m', first_purchase_month) AS month_started,
    month_passed,
    COUNT(DISTINCT fullVisitorId) AS customer_count
FROM
    first_last_purchase
GROUP BY
    month_started, month_passed
ORDER BY
    month_started, month_passed
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_cohort = query_job.to_dataframe()

# 결과 출력
df_cohort

,month_started,month_passed,customer_count
0,2016-08,0,968
1,2016-08,1,34
2,2016-08,2,13
3,2016-08,3,8
4,2016-08,4,9
...,...,...,...
66,2017-05,1,22
67,2017-05,2,16
68,2017-06,0,776
69,2017-06,1,37


피벗 테이블 생성

In [ ]:
pivot_df = df_cohort.pivot(index='month_started', columns='month_passed')

pivot_df.loc["2016-08", ("customer_count", 8)] = 0
pivot_df.loc["2016-09", ("customer_count", 5)] = 0
pivot_df.loc["2016-09", ("customer_count", 8)] = 0
pivot_df.loc["2016-09", ("customer_count", 10)] = 0
pivot_df.loc["2016-12", ("customer_count", 4)] = 0
pivot_df.loc["2017-01", ("customer_count", 6)] = 0

pivot_df

customer_count                                                  \
month_passed              0     1     2     3     4     5     6     7     8    
month_started                                                                  
2016-08                  968    34    13     8     9     5     3     3     0   
2016-09                  705    24     7     5     2     0     1     2     0   
2016-10                  694    27    14     7     5     5     7     5     3   
2016-11                  737    30    17     2     7     1     4     2     4   
2016-12                 1179    20    14    11     0     4     2     3  <NA>   
2017-01                  559    15     9     2     4     2     0  <NA>  <NA>   
2017-02                  557    19    14     7     3     6  <NA>  <NA>  <NA>   
2017-03                  679    29     5    10     3  <NA>  <NA>  <NA>  <NA>   
2017-04                  747    35    11     3  <NA>  <NA>  <NA>  <NA>  <NA>   
2017-05                  929    22    16  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>   
2017-06                  776    37  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>   
2017-07                  890  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>   

                           
month_passed     9     10  
month_started              
2016-08           2     1  
2016-09           1     0  
2016-10           2  <NA>  
2016-11        <NA>  <NA>  
2016-12        <NA>  <NA>  
2017-01        <NA>  <NA>  
2017-02        <NA>  <NA>  
2017-03        <NA>  <NA>  
2017-04        <NA>  <NA>  
2017-05        <NA>  <NA>  
2017-06        <NA>  <NA>  
2017-07        <NA>  <NA>

비율로 변경

In [ ]:
customer_cohort = pivot_df.div(pivot_df.iloc[:, 0], axis=0)*100
customer_cohort = customer_cohort.round(decimals=2)
customer_cohort = customer_cohort.loc[:"2017-06", ("customer_count", slice(1, None))]
customer_cohort

customer_count                                                  \
month_passed              1     2     3     4     5     6     7     8     9    
month_started                                                                  
2016-08                 3.51  1.34  0.83  0.93  0.52  0.31  0.31   0.0  0.21   
2016-09                  3.4  0.99  0.71  0.28   0.0  0.14  0.28   0.0  0.14   
2016-10                 3.89  2.02  1.01  0.72  0.72  1.01  0.72  0.43  0.29   
2016-11                 4.07  2.31  0.27  0.95  0.14  0.54  0.27  0.54  <NA>   
2016-12                  1.7  1.19  0.93   0.0  0.34  0.17  0.25  <NA>  <NA>   
2017-01                 2.68  1.61  0.36  0.72  0.36   0.0  <NA>  <NA>  <NA>   
2017-02                 3.41  2.51  1.26  0.54  1.08  <NA>  <NA>  <NA>  <NA>   
2017-03                 4.27  0.74  1.47  0.44  <NA>  <NA>  <NA>  <NA>  <NA>   
2017-04                 4.69  1.47   0.4  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>   
2017-05                 2.37  1.72  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>   
2017-06                 4.77  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>   

                     
month_passed     10  
month_started        
2016-08         0.1  
2016-09         0.0  
2016-10        <NA>  
2016-11        <NA>  
2016-12        <NA>  
2017-01        <NA>  
2017-02        <NA>  
2017-03        <NA>  
2017-04        <NA>  
2017-05        <NA>  
2017-06        <NA>

### 재방문율 분석

2016년 12월 코호트가 한 달차 재방문율이 급락한 이유가 뭘지?
* 유입경로 및 방문매체별 리텐션 확인
* 유입경로 및 방문매체 변화 확인
* 캠페인별 리텐션 확인 후 캠페인 진행 여부 확인

In [ ]:
# 월별 세션 수 쿼리 작성
query = f"""
SELECT
    FORMAT_DATE('%Y-%m', PARSE_DATE('%Y%m', SUBSTR(_TABLE_SUFFIX, 1, 6))) AS date,
    COUNT(*) AS total_users,
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
GROUP BY
    date
ORDER BY
    date
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_date = query_job.to_dataframe()

# 결과 출력
df_date

,date,total_users
0,2016-08,74759
1,2016-09,71032
2,2016-10,97506
3,2016-11,113972
4,2016-12,79124
5,2017-01,64694
6,2017-02,62192
7,2017-03,69931
8,2017-04,67126
9,2017-05,65371


In [ ]:
# 유입경로별 재방문율 확인 쿼리 작성
query = f"""
WITH user_sessions AS (
  SELECT
      fullVisitorId,
      MIN(PARSE_DATE('%Y%m%d', _TABLE_SUFFIX)) AS first_date,
      ANY_VALUE(trafficSource.source) AS source,
      COUNT(*) AS session_count
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      fullVisitorId
)
SELECT
    source,
    COUNT(*) AS total_users,
    COUNTIF(session_count > 1) AS retained_users,
    ROUND(COUNTIF(session_count > 1) / COUNT(*), 2) AS retention_rate
FROM
    user_sessions
GROUP BY
    source
ORDER BY
    total_users DESC,
    retention_rate DESC
LIMIT 50
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_rtn_inf = query_job.to_dataframe()

# 결과 출력
df_rtn_inf

,source,total_users,retained_users,retention_rate
0,(direct),265065,48450,0.18
1,youtube.com,200426,7600,0.04
2,google,190525,26688,0.14
3,Partners,12502,2040,0.16
4,analytics.google.com,10383,2696,0.26
5,google.com,4112,351,0.09
6,baidu,3120,169,0.05
7,m.facebook.com,3070,244,0.08
8,dfa,2410,1258,0.52
9,siliconvalley.about.com,1829,183,0.10


In [ ]:
# 방문매체별 재방문율 확인 쿼리 작성
query = f"""
WITH user_sessions AS (
  SELECT
      fullVisitorId,
      MIN(PARSE_DATE('%Y%m%d', _TABLE_SUFFIX)) AS first_date,
      ANY_VALUE(trafficSource.medium) AS medium,
      COUNT(*) AS session_count
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      fullVisitorId
)
SELECT
    medium,
    COUNT(*) AS total_users,
    COUNTIF(session_count > 1) AS retained_users,
    ROUND(COUNTIF(session_count > 1) / COUNT(*), 2) AS retention_rate
FROM
    user_sessions
GROUP BY
    medium
ORDER BY
    total_users DESC,
    retention_rate DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_rtn_med = query_job.to_dataframe()

# 결과 출력
df_rtn_med

,medium,total_users,retained_users,retention_rate
0,(none),265209,48596,0.18
1,referral,235881,14215,0.06
2,organic,187572,24540,0.13
3,affiliate,12491,2033,0.16
4,cpc,8310,2386,0.29
5,cpm,2761,1384,0.50
6,(not set),48,31,0.65


In [ ]:
# 캠페인별 재방문율 확인 쿼리 작성
query = f"""
WITH user_sessions AS (
  SELECT
      fullVisitorId,
      MIN(PARSE_DATE('%Y%m%d', _TABLE_SUFFIX)) AS first_date,
      ANY_VALUE(trafficSource.campaign) AS campaign,
      COUNT(*) AS session_count
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      fullVisitorId
)
SELECT
    campaign,
    COUNT(*) AS total_users,
    COUNTIF(session_count > 1) AS retained_users,
    ROUND(COUNTIF(session_count > 1) / COUNT(*), 2) AS retention_rate
FROM
    user_sessions
GROUP BY
    campaign
ORDER BY
    total_users DESC,
    retention_rate DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_rtn_cpn = query_job.to_dataframe()

df_rtn_cpn.to_csv('rtn_cpn.csv', index=False)

# 결과 출력
df_rtn_cpn

,campaign,total_users,retained_users,retention_rate
0,(not set),692112,89016,0.13
1,Data Share Promo,12511,2054,0.16
2,AW - Dynamic Search Ads Whole Site,4083,1240,0.30
3,AW - Accessories,3509,857,0.24
4,AW - Electronics,26,13,0.50
5,AW - Apparel,26,5,0.19
6,All Products,4,0,0.00
7,Data Share,1,0,0.00


In [ ]:
# 월별 유입경로 변화 확인 쿼리 작성
query = f"""
SELECT
    FORMAT_DATE('%Y-%m', PARSE_DATE('%Y%m', SUBSTR(_TABLE_SUFFIX, 1, 6))) AS date,
    trafficSource.source AS source,
    COUNT(*) AS visit_count
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
    AND totals.transactionRevenue > 0
GROUP BY
    date,
    trafficSource.source
ORDER BY
    date,
    visit_count DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_inf = query_job.to_dataframe()

df_inf.to_csv('inf.csv', index=False)

# 결과 출력
df_inf

,date,source,visit_count
0,2016-08,(direct),733
1,2016-08,google,346
2,2016-08,dfa,13
3,2016-08,sites.google.com,6
4,2016-08,facebook.com,5
...,...,...,...
174,2017-07,bing,2
175,2017-07,youtube.com,1
176,2017-07,m.facebook.com,1
177,2017-07,outlook.live.com,1


In [ ]:
# 월별 방문매체 변화 확인 쿼리 작성
query = f"""
SELECT
    FORMAT_DATE('%Y-%m', PARSE_DATE('%Y%m', SUBSTR(_TABLE_SUFFIX, 1, 6))) AS date,
    trafficSource.medium AS medium,
    COUNT(*) AS visit_count
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
    AND totals.transactionRevenue > 0
GROUP BY
    date,
    medium
ORDER BY
    date,
    visit_count DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_med = query_job.to_dataframe()

df_med.to_csv('med.csv', index=False)

# 결과 출력
df_med

,date,medium,visit_count
0,2016-08,(none),733
1,2016-08,organic,321
2,2016-08,cpc,28
3,2016-08,referral,22
4,2016-08,cpm,13
...,...,...,...
61,2017-07,organic,296
62,2017-07,cpc,45
63,2017-07,cpm,19
64,2017-07,referral,14


In [ ]:
# 월별 캠페인 변화 확인 쿼리 작성
query = f"""
SELECT
    FORMAT_DATE('%Y-%m', PARSE_DATE('%Y%m', SUBSTR(_TABLE_SUFFIX, 1, 6))) AS date,
    trafficSource.campaign AS campaign,
    COUNT(*) AS visit_count
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
    AND totals.transactionRevenue > 0
GROUP BY
    date,
    campaign
ORDER BY
    date,
    visit_count DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_cpn = query_job.to_dataframe()

df_cpn.to_csv('cpn.csv', index=False)

# 결과 출력
df_cpn

,date,campaign,visit_count
0,2016-08,(not set),1092
1,2016-08,AW - Dynamic Search Ads Whole Site,25
2,2016-08,Data Share Promo,2
3,2016-09,(not set),834
4,2016-09,AW - Dynamic Search Ads Whole Site,25
5,2016-10,(not set),861
6,2016-10,AW - Dynamic Search Ads Whole Site,10
7,2016-10,Data Share Promo,1
8,2016-11,(not set),917
9,2016-11,AW - Dynamic Search Ads Whole Site,2


id, month, source, median, campaign

In [ ]:
# 월별 캠페인 변화 확인 쿼리 작성
query = f"""
SELECT
    FORMAT_DATE('%Y-%m', PARSE_DATE('%Y%m', SUBSTR(_TABLE_SUFFIX, 1, 6))) AS date,
    fullVisitorId AS id,
    trafficSource.source AS source,
    trafficSource.medium AS medium,
    trafficSource.campaign AS campaign
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
GROUP BY
    date, id, source, medium, campaign
ORDER BY
    date,
    fullVisitorId
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df = query_job.to_dataframe()

# 결과 출력
df

,date,id,source,medium,campaign
0,2016-08,000005103959234087,google,organic,(not set)
1,2016-08,0000114156543135683,youtube.com,referral,(not set)
2,2016-08,0000253133284033056,youtube.com,referral,(not set)
3,2016-08,0000393882583300244,youtube.com,referral,(not set)
4,2016-08,0000606911577080749,youtube.com,referral,(not set)
...,...,...,...,...,...
768432,2017-07,9998955356642683968,google,organic,(not set)
768433,2017-07,9999379202747789695,google,organic,(not set)
768434,2017-07,9999430912254902219,google,organic,(not set)
768435,2017-07,9999594748540438502,(direct),(none),(not set)


## DAU/MAU

* DAU (Daily Active Users, 일간 활성 사용자): 하루 동안 서비스를 이용한 고유 사용자의 수

* MAU (Monthly Active Users, 월간 활성 사용자): 한 달 동안 서비스를 이용한 고유 사용자의 수

* DAU/MAU 비율 (사용자 고착도, Stickiness): 월간 활성 사용자 중 얼마나 많은 사용자가 매일 서비스에 접속하는지를 보여줌

In [ ]:
# DAU/MAU 쿼리 작성
query = f"""
#standardSQL
WITH
daily AS (
  SELECT
      date,
      COUNT(DISTINCT fullVisitorId) AS DAU
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      date
),
monthly AS (
  SELECT
      SUBSTR(date, 1, 6) AS month,
      COUNT(DISTINCT fullVisitorId) AS MAU
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170731'
  GROUP BY
      month
)
SELECT
    d.date,
    d.DAU,
    m.MAU,
    ROUND(d.DAU / m.MAU, 2) AS DAU_MAU_ratio
FROM
    daily d
JOIN
    monthly m
  ON SUBSTR(d.date, 1, 6) = m.month
ORDER BY
    d.date
"""
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = False

query_job = client.query(query, job_config=job_config)
df_dau_mau = query_job.to_dataframe()

df_dau_mau.to_csv('dau_mau.csv', index=False)

df_dau_mau

,date,DAU,MAU,DAU_MAU_ratio
0,20160801,1569,61699,0.03
1,20160802,1961,61699,0.03
2,20160803,2657,61699,0.04
3,20160804,2947,61699,0.05
4,20160805,2492,61699,0.04
...,...,...,...,...
360,20170727,2297,58569,0.04
361,20170728,2233,58569,0.04
362,20170729,1498,58569,0.03
363,20170730,1649,58569,0.03


* 주중에 비해 주말의 DAU 수치가 현저히 낮음

# 4. Revenue (수익)

**의미**
* 고객이 제품이나 서비스에 대해 비용을 지불하는 단계
* 비즈니스의 수익 창출에 직접적으로 연결됨
<br>

**목표**
* 고객의 사용 및 경험을 통해 실제 수익을 창출
* 수익 모델(구독, 인앱 구매, 광고 등)을 최적화
<br>

**KPI**
* 총 수익
* 평균 고객 수익 (ARPU,  Average Revenue Per User)
* 재구매율 (RPR, Repeat Purchase Rate)
* 고객 생애 가치 (LTV, Lifetime Value)
<br>

**분석 주제**
* 채널별 수익 분석
* RFM 분석 - 고객 세분화 (각 그룹의 ARPU, RPR, LTV 등을 분석)

## 방문당 평균 수익 (RPV)

### (1) 유입 경로

In [ ]:
# 유입 경로별 방문당 평균 수익
query = f"""
SELECT
    trafficSource.source AS source,
    count(*) AS visit_count,
    SUM(totals.transactionRevenue) / 1000000 AS total_revenue, -- 해당 변수는 마이크로 단위이므로 1,000,000으로 나눠야 함
    SUM(totals.transactionRevenue) / 1000000 / COUNT(*) AS rpv -- 방문당 평균 수익(Revenue per Visit, RPV)
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND totals.transactionRevenue > 0
GROUP BY
    source
ORDER BY
    rpv DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_inflow_RPV = query_job.to_dataframe()

# 결과 출력
df_inflow_RPV

,source,visit_count,total_revenue,rpv
0,dfa,123,76928.42,625.434309
1,mail.google.com,62,23327.91,376.256613
2,dealspotr.com,40,5690.64,142.266000
3,(direct),8713,1190175.11,136.597625
4,connect.googleforwork.com,3,404.81,134.936667
5,basecamp.com,1,105.80,105.800000
6,sites.google.com,42,4390.78,104.542381
7,search.myway.com,1,100.94,100.940000
8,google,2334,229022.49,98.124460
9,mg.mail.yahoo.com,4,377.31,94.327500


### (2) 캠페인

In [ ]:
# 캠페인별 세션 수 분석 쿼리 작성
query = """
SELECT
    trafficSource.campaign AS campaign,
    COUNT(*) AS visit_count,
    SUM(totals.transactionRevenue) / 1000000 AS total_revenue, -- 해당 변수는 마이크로 단위이므로 1,000,000으로 나눠야 함
    SUM(totals.transactionRevenue) / 1000000 / COUNT(*) AS rpv -- 방문당 평균 수익(Revenue per Visit, RPV)
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND totals.transactionRevenue > 0
GROUP BY
    campaign
ORDER BY
    rpv DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_campaign_RPV = query_job.to_dataframe()

# 결과 출력
df_campaign_RPV

,campaign,visit_count,total_revenue,rpv
0,AW - Accessories,98,13997.55,142.832143
1,(not set),11271,1514688.95,134.388160
2,AW - Dynamic Search Ads Whole Site,137,10787.36,78.739854
3,Data Share Promo,9,597.38,66.375556


### (3) 방문 매체

In [ ]:
# 방문 매체별 수익 분석 쿼리 작성
query = """
SELECT
    trafficSource.medium AS medium,
    COUNT(*) AS visit_count,
    SUM(totals.transactionRevenue) / 1000000 AS total_revenue, -- 해당 변수는 마이크로 단위이므로 1,000,000으로 나눠야 함
    SUM(totals.transactionRevenue) / 1000000 / COUNT(*) AS rpv -- 방문당 평균 수익(Revenue per Visit, RPV)
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND totals.transactions > 0
GROUP BY
    medium
ORDER BY
    rpv DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_medium = query_job.to_dataframe()

df_medium.to_csv('RPV_medium.csv', index=False)

# 결과 출력
df_medium

,medium,visit_count,total_revenue,rpv
0,cpm,141,78268.19,555.093546
1,referral,290,40954.76,141.223310
2,(none),8742,1190175.11,136.144488
3,cpc,243,25176.76,103.608066
4,organic,2126,204889.05,96.373024
5,affiliate,9,597.38,66.375556
6,(not set),1,9.99,9.990000


### (4) 국가

In [ ]:
# 국가별 수익 분석 쿼리 작성
query = """
  SELECT
      geoNetwork.country AS country,
      COUNT(*) AS visit_count,
      SUM(totals.transactionRevenue) / 1000000 AS total_revenue, -- 해당 변수는 마이크로 단위이므로 1,000,000으로 나눠야 함
      SUM(totals.transactionRevenue) / 1000000 / COUNT(*) AS rpv -- 방문당 평균 수익(Revenue per Visit, RPV)
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
      AND totals.transactionRevenue IS NOT NULL
  GROUP BY
      country
  ORDER BY
      rpv DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_country_RPV = query_job.to_dataframe()

df_country_RPV.to_csv('RPV_country.csv', index=False)

# 결과 출력
df_country_RPV

,country,visit_count,total_revenue,rpv
0,Kenya,3,5268.70,1756.233333
1,Nigeria,2,3302.40,1651.200000
2,Japan,17,6728.99,395.822941
3,Portugal,1,215.80,215.800000
4,Venezuela,63,13374.90,212.300000
...,...,...,...,...
64,Russia,4,94.94,23.735000
65,Georgia,2,38.98,19.490000
66,Kazakhstan,2,34.98,17.490000
67,Anguilla,1,10.99,10.990000


## RFM 분석

**RFM 분석**

고객의 구매 행동을 기반으로 충성도 높은 고객을 식별하고 마케팅 전략을 최적화하는 데 사용되는 데이터 분석 기법

<br>

RFM 분석의 세 가지 요소

* Recency (최신성): 현재날짜와 고객이 마지막으로 구매한 날짜의 차이
* Frequency (빈도): 고객이 특정 기간 동안 몇 번 구매했는지 (구매 횟수)
* Monetary (금액): 고객이 특정 기간 동안 총 얼마를 소비했는지 (구매 금액)

### R, F, M 변수 생성

In [ ]:
# RFM 변수 생성 쿼리 작성
query = f"""
SELECT
    fullVisitorId,
    DATE_DIFF(
        PARSE_DATE('%Y%m%d', '20170802'),
        MAX(PARSE_DATE('%Y%m%d', TRIM(_TABLE_SUFFIX))),  -- 앞뒤에 공백이 포함된 문자형이 있는 것으로 추정. TRIM 사용
        DAY
    ) AS recency,
    COUNT(*) AS frequency,
    SUM(totals.transactionRevenue) / 1000000 AS monetary -- 마이크로 단위이므로 변환
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    TRIM(_TABLE_SUFFIX) BETWEEN '20160801' AND '20170801'
    AND totals.transactions > 0
GROUP BY
    fullVisitorId
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_rfm = query_job.to_dataframe()

# 결과 출력
df_rfm

,fullVisitorId,recency,frequency,monetary
0,0548744394505098293,8,1,32.78
1,1424355572088327868,8,1,99.99
2,963845761761571092,8,1,20.78
3,7352817445808452606,351,1,43.44
4,7478999854159501077,351,1,48.28
...,...,...,...,...
10017,500766381427960566,140,1,50.37
10018,7942338055212909628,140,1,52.49
10019,2234644290294742127,162,1,34.99
10020,7800508396890862296,326,1,42.85


### R, F, M 점수 설정 및 고객 세분화

 * Recency (최근성) 점수: 최근 방문한 고객이 높은 점수(5)
 * Frequency (빈도) 점수: 자주 방문한 고객이 높은 점수(1 or 2이상)
 * Monetary (구매 금액) 점수: 많이 구매한 고객이 높은 점수(5)


R,F,M 점수 설정

In [ ]:
# R 값의 경우 수치가 커질수록 낮은 점수 부여 (수치가 크면 구매한 지 오래 됐다는 의미)
r_labels = list(range(5, 0, -1)) # Recency (최근성) 점수: 최근 방문한 고객이 높은 점수(5)
m_labels = list(range(1,6))      # Monetary (구매 금액) 점수: 많이 구매한 고객이 높은 점수(5)

interval = 5

# r_cut: rfm_df['recency']를 같은 크기로 interval 개의 구간으로 나누는데 구간별 이름은 r_labels로 한다.
r_cut = pd.qcut(df_rfm['recency'], interval, labels = r_labels)
m_cut = pd.qcut(df_rfm['monetary'], interval, labels = m_labels)

# rfm_df에 각각의 점수 할당
rfm_df = df_rfm.assign(R_score = r_cut,
                       F_score = np.where(df_rfm['frequency'] >= 2, 2, 1),
                       M_score = m_cut)

# monetary에 26개의 결측치 존재 -> 제거
rfm_df = rfm_df.dropna()
rfm_df

,fullVisitorId,recency,frequency,monetary,R_score,F_score,M_score
0,0548744394505098293,8,1,32.78,5,1,2
1,1424355572088327868,8,1,99.99,5,1,4
2,963845761761571092,8,1,20.78,5,1,2
3,7352817445808452606,351,1,43.44,1,1,3
4,7478999854159501077,351,1,48.28,1,1,3
...,...,...,...,...,...,...,...
10017,500766381427960566,140,1,50.37,3,1,3
10018,7942338055212909628,140,1,52.49,3,1,3
10019,2234644290294742127,162,1,34.99,3,1,2
10020,7800508396890862296,326,1,42.85,1,1,3


고객 세분화

In [ ]:
def classify_customer_segment(row):
  R, F, M = row['R_score'], row['F_score'], row['M_score']

  if R == 5 and F == 2 and M == 5:
    return 'Champions'

  elif R >= 3 and F >= 1 and M >= 4:
    return 'Loyal Customers'

  elif R >= 2 and F >= 1 and M >= 1:
    return 'At Risk'

  elif R == 1 and F >= 1 and M >= 1:
    return 'Lost'

rfm_df['Segment'] = rfm_df.apply(classify_customer_segment, axis=1)
rfm_df

,fullVisitorId,recency,frequency,monetary,R_score,F_score,M_score,Segment
0,0548744394505098293,8,1,32.78,5,1,2,At Risk
1,1424355572088327868,8,1,99.99,5,1,4,Loyal Customers
2,963845761761571092,8,1,20.78,5,1,2,At Risk
3,7352817445808452606,351,1,43.44,1,1,3,Lost
4,7478999854159501077,351,1,48.28,1,1,3,Lost
...,...,...,...,...,...,...,...,...
10017,500766381427960566,140,1,50.37,3,1,3,At Risk
10018,7942338055212909628,140,1,52.49,3,1,3,At Risk
10019,2234644290294742127,162,1,34.99,3,1,2,At Risk
10020,7800508396890862296,326,1,42.85,1,1,3,Lost


In [ ]:
# 각 세그먼트의 비율
rfm_df['Segment'].value_counts(normalize=True).to_csv('rfm_segment_ratio.csv')

In [ ]:
rfm_df['Segment'].value_counts(normalize=True)

,proportion
Segment,
At Risk,0.565126
Loyal Customers,0.223689
Lost,0.197079
Champions,0.014106


## 고객 세그먼트별 ARPU, RPR, LTV 분석

* 사용자당 평균 수익 (ARPU, Average Revenue Per User) = 총 매출액 / 총 고객 수
* 재구매율 (RPR, Repeat Purchase Rate) = 구매 고객 수 / 두 번 이상 구매한 고객 수
* 고객 생애 가치 (LTV, Customer Lifetime Value) = ARPU * 고객 평균 생애 기간 (고객이 평생 창출할 수 있는 총 가치를 추정)

고객 평균 생애 기간은 고객별로 첫 구매일과 마지막 구매일의 날짜 차이를 계산한 후 세그먼트별로 평균을 낸 값으로 정의

In [ ]:
# duckdb는 ga_sessions_와 같은 와일드카드 테이블을 참조할 수 없으므로 해당 테이블을 대체할 임시 데이터프레임 생성
query = f"""
  SELECT
      fullVisitorId,
      DATE_DIFF(
        MAX(PARSE_DATE('%Y%m%d', TRIM(_TABLE_SUFFIX))),
        MIN(PARSE_DATE('%Y%m%d', TRIM(_TABLE_SUFFIX))),
        DAY
      ) AS date_diff
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      TRIM(_TABLE_SUFFIX) BETWEEN '20160801' AND '20170801'
      AND totals.transactions > 0
  GROUP BY
      fullVisitorId
  ORDER BY
      fullVisitorId
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_datediff = query_job.to_dataframe()

df_datediff['date_diff'] = df_datediff['date_diff'] + np.repeat(1, 10022)

# 결과 출력
df_datediff

,fullVisitorId,date_diff
0,0000213131142648941,1
1,0002871498069867123,1
2,0003450834640354121,1
3,0003961110741104601,1
4,000435324061339869,1
...,...,...
10017,9991633376050115277,1
10018,9994767073213036303,1
10019,9997409246962677759,1
10020,9998597322098588317,1


In [ ]:
# 고객 세그먼트별 ARPU, 재구매율, LTV 계산 쿼리 작성
import duckdb

query = f"""
WITH
RE AS ( -- 2번 이상 구매한 고객 수의 세그먼트별 합계
  SELECT
      Segment,
      COUNT(*) AS count_over_twice
  FROM
      rfm_df
  WHERE
      F_score > 1
  GROUP BY
      Segment
)
SELECT
    rfm.Segment,
    -- 세그먼트 내 고객당 평균 구매금액 (ARPU)
    SUM(rfm.monetary) / COUNT(df.fullVisitorId) AS ARPU,
    -- 재구매율: (2번 이상 구매한 고객 수) / (해당 세그먼트의 고객 수)
    RE.count_over_twice / COUNT(df.fullVisitorId) AS RPR,
    -- LTV: ARPU * 평균 고객 생애기간(일)
    (SUM(rfm.monetary) / COUNT(df.fullVisitorId)) * AVG(df.date_diff) AS LTV
FROM
    rfm_df AS rfm
  JOIN df_datediff AS df USING (fullVisitorId)
  LEFT JOIN RE ON rfm.Segment = RE.Segment
GROUP BY
    rfm.Segment,
    RE.count_over_twice
"""

df_segment = duckdb.query(query).to_df()
# One-Time Customers은 재구매한 고객이 없으므로 결측치가 할당되는데, 이것을 0으로 대체한다.
df_segment = df_segment.fillna(0)

df_segment

,Segment,ARPU,RPR,LTV
0,At Risk,63.882146,0.048504,168.938816
1,Champions,1629.497660,1.000000,99561.151328
2,Loyal Customers,294.925519,0.182916,2931.844558
3,Lost,147.202218,0.073604,283.793921


# 5. Referral (추천)

**의미** <br>
기존 고객이 자신의 경험을 바탕으로 친구, 가족, 지인 등에게 제품이나 서비스를 추천하는 행위

<br>

**목표** <br>
* 고객 간의 입소문을 통해 신규 고객을 유입시킴으로써 지속적이고 자연스러운 성장을 도모
* 광고나 직접적인 마케팅 비용 없이 추천을 통한 고객 확보로 비용 효율성 높임
* 추천 유입 고객이 비추천 유입 고객보다 더 높은 전환율과 매출을 발생시키는지 t-test 수행


## 추천/비추천 유입 고객 트래픽과 수익 비교 분석

In [ ]:
# 추천 유입 고객 트래픽 분석 쿼리
query = f"""
SELECT
    COUNT(*) AS total_sessions,
    SUM(IF(totals.pageviews = 1, 1, 0)) AS bounced_sessions,
    SUM(IF(totals.pageviews = 1, 1, 0)) / COUNT(*) AS bounce_rate,
    AVG(totals.timeOnSite) AS avg_session_duration,
    AVG(totals.pageviews) AS avg_pageviews,
    SUM(totals.transactionRevenue) / 1000000 AS total_revenue, -- 해당 변수는 마이크로 단위이므로 1,000,000으로 나눠야 함
    SUM(totals.transactionRevenue) / 1000000 / COUNT(*) AS rpv -- 방문당 평균 수익(RPV, Revenue per Visit)
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    trafficSource.medium = 'referral'
    AND _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND totals.transactionRevenue > 0
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_referral_traffic = query_job.to_dataframe()

# 결과 출력
df_referral_traffic

,total_sessions,bounced_sessions,bounce_rate,avg_session_duration,avg_pageviews,total_revenue,rpv
0,289,0,0.0,1138.235294,31.079585,40954.76,141.711972


In [ ]:
# 비추천 유입 고객 트래픽 분석 쿼리
query = f"""
SELECT
    COUNT(*) AS total_sessions,
    SUM(IF(totals.pageviews = 1, 1, 0)) AS bounced_sessions,
    SUM(IF(totals.pageviews = 1, 1, 0)) / COUNT(*) AS bounce_rate,
    AVG(totals.timeOnSite) AS avg_session_duration,
    AVG(totals.pageviews) AS avg_pageviews,
    SUM(totals.transactionRevenue) / 1000000 AS total_revenue, -- 해당 변수는 마이크로 단위이므로 1,000,000으로 나눠야 함
    SUM(totals.transactionRevenue) / 1000000 / COUNT(*) AS rpv -- 방문당 평균 수익(Revenue per Visit, RPV)
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    trafficSource.medium != 'referral'
    AND _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND totals.transactionRevenue > 0
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_not_referral_traffic = query_job.to_dataframe()

# 결과 출력
df_not_referral_traffic

,total_sessions,bounced_sessions,bounce_rate,avg_session_duration,avg_pageviews,total_revenue,rpv
0,11226,0,0.0,1061.046244,28.259665,1499116.48,133.539683


t-test 수행

In [ ]:
query = f"""
SELECT
    trafficSource.medium AS medium,
    totals.totalTransactionRevenue / 1000000 AS revenue,  -- 수익 (단위 변환)
    fullVisitorId -- 방문 ID (세션)
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND totals.transactionRevenue > 0
"""

# 쿼리 실행
query_job = client.query(query)
df_tmp = query_job.to_dataframe()

# 데이터 확인
df_tmp

,medium,revenue,fullVisitorId
0,(none),78.99,1166512440754294528
1,(none),26.99,8434622228747679141
2,(none),61.07,4008726419400228329
3,(none),104.95,4271792018121577582
4,(none),22.19,6047640651761300200
...,...,...,...
11510,referral,55.24,4242693912596237127
11511,(none),57.18,8151988731272450660
11512,organic,63.97,1790630358090197146
11513,(none),996.12,9026840718082010040


In [ ]:
import scipy.stats as stats

# Referral vs Non-Referral 그룹화
df_referral = df_tmp[df_tmp['medium'] == 'referral']
df_non_referral = df_tmp[df_tmp['medium'] != 'referral']

# 방문당 평균 수익(RPV) 계산
df_referral_rpv = df_referral.groupby('fullVisitorId')['revenue'].sum()
df_non_referral_rpv = df_non_referral.groupby('fullVisitorId')['revenue'].sum()

# t-검정 수행 (독립표본 t-검정)
t_stat, p_value = stats.ttest_ind(df_referral_rpv, df_non_referral_rpv, equal_var=False)  # Welch's t-test

# 결과 출력
print(f"t-통계량: {t_stat:.4f}")
print(f"p-값: {p_value:.4f}")

t-통계량: 0.0589
p-값: 0.9531
추천 고객과 비추천 고객의 방문당 평균 수익(RPV)에 통계적으로 유의미한 차이가 없음 (귀무가설 채택)



## 추천/ 비추천 퍼널 분석

### 각 단계별 세션 수 산출

In [ ]:
# 추천 유입 유저들의 단계별 세션 수 분류 쿼리 작성
query = f"""
SELECT
    h.eCommerceAction.action_type AS action,
    COUNT(*) AS count
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS h
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND h.eCommerceAction.action_type IN ('1', '2', '3', '5', '6')
    AND trafficSource.medium = 'referral'
GROUP BY
    action
ORDER BY
    count DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_ref_funnels = query_job.to_dataframe()

# action 명칭 수정
df_ref_funnels['action'] = ['Product list', 'Product detail page', 'Add to cart', 'Check out', 'Completed purchase']

df_ref_funnels.to_csv('ref_funnels.csv', index=False)

# 결과 출력
df_ref_funnels

,action,count
0,Product list,30079
1,Product detail page,25639
2,Add to cart,6803
3,Check out,4190
4,Completed purchase,662


In [ ]:
# 비추천 유입 유저들의 단계별 세션 수 분류 쿼리 작성
query = f"""
SELECT
    h.eCommerceAction.action_type AS action,
    COUNT(*) AS count
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS h
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND h.eCommerceAction.action_type IN ('1', '2', '3', '5', '6')
    AND trafficSource.medium != 'referral'
GROUP BY
    action
ORDER BY
    count DESC
"""

# 쿼리 실행 및 데이터 가져오기
query_job = client.query(query)
df_not_ref_funnels = query_job.to_dataframe()

# action 명칭 수정
df_not_ref_funnels['action'] = ['Product list', 'Product detail page', 'Add to cart', 'Check out', 'Completed purchase']

df_not_ref_funnels.to_csv('not_ref_funnels.csv', index=False)

# 결과 출력
df_not_ref_funnels

,action,count
0,Product list,415681
1,Product detail page,336968
2,Add to cart,114624
3,Check out,83829
4,Completed purchase,23870


### 전환율(Conversion Rate), 이탈률(Bounce rate)

In [ ]:
conversion_rate = [100]

for i in range(0,4):
  conversion_rate.append(round((df_ref_funnels['count'].iloc[i+1]/df_ref_funnels['count'].iloc[i])*100, 2))

df_ref_funnels['conversion rate'] = conversion_rate
df_ref_funnels['Bounce rate'] = np.repeat(100, 5) - np.array(conversion_rate)

df_ref_funnels.to_csv('ref_funnels_con_bounce.csv', index=False)

df_ref_funnels

,action,count,conversion rate,Bounce rate
0,Product list,30079,100.00,0.00
1,Product detail page,25639,85.24,14.76
2,Add to cart,6803,26.53,73.47
3,Check out,4190,61.59,38.41
4,Completed purchase,662,15.80,84.20


In [ ]:
conversion_rate = [100]

for i in range(0,4):
  conversion_rate.append(round((df_not_ref_funnels['count'].iloc[i+1]/df_not_ref_funnels['count'].iloc[i])*100, 2))

df_not_ref_funnels['conversion rate'] = conversion_rate
df_not_ref_funnels['Bounce rate'] = np.repeat(100, 5) - np.array(conversion_rate)

df_not_ref_funnels.to_csv('not_ref_funnels_con_bounce.csv', index=False)

df_not_ref_funnels

,action,count,conversion rate,Bounce rate
0,Product list,415681,100.00,0.00
1,Product detail page,336968,81.06,18.94
2,Add to cart,114624,34.02,65.98
3,Check out,83829,73.13,26.87
4,Completed purchase,23870,28.47,71.53


* 병목구간 분석
* 추천과 비추천 퍼널 비교